# Tutorial: The Basic Tools of Private Deep Learning -- TensorFlow edition!


**Note that this tutorial was originally designed part of [OpenMined PySyft-TensorFlow tutorials](https://github.com/OpenMined/PySyft-TensorFlow/tree/master/examples)**

Welcome to PySyft's introductory tutorial for privacy preserving, decentralized deep learning. This series of notebooks is a step-by-step guide for you to get to know the new tools and techniques required for doing deep learning on secret/private data/models without centralizing them under one authority.

# Part 0: Hook TensorFlow

To add PySyft functionalities to TensorFlow, you just have to call `hook = sy.TensorFlowHook(tf)`.

In [1]:
# Run this cell to see if things work
import tensorflow as tf
import syft as sy
hook = sy.TensorFlowHook(tf)

tf.constant([1, 2, 3, 4, 5.])

<tf.Tensor: id=4, shape=(5,), dtype=float32, numpy=array([1., 2., 3., 4., 5.], dtype=float32)>

If this cell executed, then you're off to the races! Let's do this!

# Part 1: The Basic Tools of Private, Decentralized Data Science

So - the first question you may be wondering is - How in the world do we train a model on data we don't have access to? 

Well, the answer is surprisingly simple. If you're used to working in TensorFlow, then you're used to working with tf.Tensor objects like these!

In [2]:
x = tf.constant([1, 2, 3, 4, 5.])
y = x + x
y

<tf.Tensor: id=6, shape=(5,), dtype=float32, numpy=array([ 2.,  4.,  6.,  8., 10.], dtype=float32)>

Obviously, using these super fancy (and powerful!) tensors is important, but also requires you to have the data on your local machine. This is where our journey begins. 

# Section 1.1 - Sending Tensors to Bob's Machine

Whereas normally we would perform data science / deep learning on the machine which holds the data, now we want to perform this kind of computation on some **other** machine. More specifically, we can no longer simply assume that the data is on our local machine.

Thus, instead of using TensorFlow tensors, we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [3]:
bob = sy.VirtualWorker(hook, id="bob")

Let's say Bob's machine is on another planet - perhaps on Mars! But, at the moment the machine is empty. Let's create some data so that we can send it to Bob and learn about pointers!

In [4]:
x = tf.constant([1., 2., 3., 4., 5.])
y = tf.constant([1., 1., 1., 1., 1.])

And now - let's send our tensors to Bob!!

In [5]:
x_ptr = x.send(bob)
y_ptr = y.send(bob)

BOOM! Now Bob has two tensors! Don't believe me? Have a look for yourself!

In [6]:
bob._objects

{41987490178: <tf.Tensor: id=11, shape=(5,), dtype=float32, numpy=array([1., 2., 3., 4., 5.], dtype=float32)>,
 72927188351: <tf.Tensor: id=15, shape=(5,), dtype=float32, numpy=array([1., 1., 1., 1., 1.], dtype=float32)>}

In [7]:
z_ptr = x_ptr + x_ptr
z_ptr

(Wrapper)>[PointerTensor | me:86379356515 -> bob:14242433809]

In [8]:
bob._objects

{41987490178: <tf.Tensor: id=11, shape=(5,), dtype=float32, numpy=array([1., 2., 3., 4., 5.], dtype=float32)>,
 72927188351: <tf.Tensor: id=15, shape=(5,), dtype=float32, numpy=array([1., 1., 1., 1., 1.], dtype=float32)>,
 14242433809: <tf.Tensor: id=17, shape=(5,), dtype=float32, numpy=array([ 2.,  4.,  6.,  8., 10.], dtype=float32)>}

Now notice something. When we called `x.send(bob)` it returned a new object that we called `x_ptr`. This is our first *pointer* to a tensor. Pointers to tensors do NOT actually hold data themselves. Instead, they simply contain metadata about a tensor (with data) stored on another machine. The purpose of these tensors is to give us an intuitive API to tell the other machine to compute functions using this tensor. Let's take a look at the metadata that pointers contain.

In [9]:
z_ptr

(Wrapper)>[PointerTensor | me:86379356515 -> bob:14242433809]

Check out that metadata!

There are two main attributes specific to pointers:

- `x_ptr.location : bob`, the location, a reference to the location that the pointer is pointing to
- `x_ptr.id_at_location : <random integer>`, the id where the tensor is stored at location

They are printed in the format `<id_at_location>@<location>`

There are also other more generic attributes:
- `x_ptr.id : <random integer>`, the id of our pointer tensor, it was allocated randomly
- `x_ptr.owner : "me"`, the worker which owns the pointer tensor, here it's the local worker, named "me"


In [10]:
x_ptr.location

<VirtualWorker id:bob #objects:3>

In [11]:
bob

<VirtualWorker id:bob #objects:3>

In [12]:
bob == x_ptr.location

True

In [13]:
x_ptr.id_at_location

41987490178

In [14]:
x_ptr.owner

<VirtualWorker id:me #objects:0>

You may wonder why the local worker which owns the pointer is also a VirtualWorker, although we didn't create it.
Fun fact, just like we had a VirtualWorker object for Bob, we (by default) always have one for us as well. This worker is automatically created when we construct `hook = sy.TensorFlowHook(tf)` and so you don't usually have to create it yourself.

In [15]:
me = sy.local_worker
me

<VirtualWorker id:me #objects:0>

In [16]:
me == x_ptr.owner

True

And finally, just like we can call .send() on a tensor, we can call .get() on a pointer to a tensor to get it back!!!

In [17]:
x_ptr.get()

<tf.Tensor: id=21, shape=(5,), dtype=float32, numpy=array([1., 2., 3., 4., 5.], dtype=float32)>

In [18]:
y_ptr.get()

<tf.Tensor: id=24, shape=(5,), dtype=float32, numpy=array([1., 1., 1., 1., 1.], dtype=float32)>

In [19]:
z_ptr.get()

<tf.Tensor: id=27, shape=(5,), dtype=float32, numpy=array([ 2.,  4.,  6.,  8., 10.], dtype=float32)>

In [20]:
bob._objects

{}

And as you can see... Bob no longer has the tensors anymore!!! They've moved back to our machine!

# Section 1.2 - Arithmetic with Tensor Pointers

So, sending and receiving tensors from Bob is great, but this is hardly Deep Learning! We want to be able to perform tensor _operations_ on remote tensors. Fortunately, tensor pointers make this quite easy! You can just use pointers like you would normal tensors!

In [21]:
x = tf.constant([1., 2., 3., 4.])
x_ptr = x.send(bob)

y_ptr = x_ptr + x_ptr

y = tf.reshape(y_ptr, [2, 2])
id = tf.constant([[1., 0.], [0., 1.]]).send(bob)

z = tf.matmul(y, id)

print(z.get())

tf.Tensor(
[[2. 4.]
 [6. 8.]], shape=(2, 2), dtype=float32)


And voilà! 

Behind the scenes, something very powerful happened. Instead of doing a matrix multiplication between x and an identity matrix locally, several commands were serialized and sent to Bob, who performed the computation remotely on his machine until we called .get() on z to get the final result.

### Variables

You can perform similar operations on `tf.Variable` as well. Here is an example of manual differentiation.

In [22]:
x = tf.expand_dims(id[0], 0)

w_init = tf.initializers.glorot_normal()
w = tf.Variable(w_init(shape=(2, 1), dtype=tf.float32)).send(bob)
z = tf.matmul(x, w)

# # Manual differentiation & update
dzdx = tf.transpose(x)
w.assign_sub(dzdx)

print("Updated: ", w.get())


Updated:  <tf.Variable 'Variable:0' shape=(2, 1) dtype=float32, numpy=
array([[-0.28401625],
       [ 0.6857992 ]], dtype=float32)>


[Automatic differentiation](https://www.tensorflow.org/tutorials/customization/autodiff) with `tf.GradientTape` is on the roadmap!


### Join OpenMined Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)